In [22]:
import threading
import time

In [12]:
print(threading.active_count())

6


In [8]:
print(threading.enumerate())

[<_MainThread(MainThread, started 29828)>, <Thread(IOPub, started daemon 28244)>, <Heartbeat(Heartbeat, started daemon 15312)>, <ControlThread(Control, started daemon 20188)>, <HistorySavingThread(IPythonHistorySavingThread, started 35316)>, <ParentPollerWindows(Thread-1, started daemon 6876)>]


In [9]:
print(threading.current_thread())

<_MainThread(MainThread, started 29828)>


In [13]:
def thread_job():
    print(f'This is a added job and the number is {threading.current_thread()}')

In [19]:
added_thread = threading.Thread(target=thread_job)

In [20]:
added_thread.start()

This is a added job and the number is <Thread(Thread-5, started 15588)>


In [21]:
added_thread.start()

RuntimeError: threads can only be started once

In [28]:
def thread_job(name):
    print(f'{name} start')
    for i in range(10):
        time.sleep(0.1)
    print(f'{name} Finish')

In [30]:
def thread_job_no_sleep(name):
    print(f'{name} start')
    print(f'{name} Finish')

In [29]:
added_thread = threading.Thread(target=thread_job,args=("T1",), name='T1')
added_thread.start()
print('All Done')

T1 start
All Done
T1 Finish


In [25]:
added_thread = threading.Thread(target=thread_job,args=("T1",), name='T1')
added_thread.start()
added_thread.join()
print('All Done')

T1 start
T1 Finish
All Done


In [31]:
thread1 = threading.Thread(target=thread_job,args=("T1",), name='T1')
thread1.start()
thread2 = threading.Thread(target=thread_job_no_sleep,args=("T2",), name='T2')
thread2.start()
print('All Done')

T1 start
T2 start
T2 Finish
All Done
T1 Finish


In [32]:
thread1 = threading.Thread(target=thread_job,args=("T1",), name='T1')
thread1.start()
thread2 = threading.Thread(target=thread_job_no_sleep,args=("T2",), name='T2')
thread2.start()
thread1.join()
print('All Done')

T1 start
T2 start
T2 Finish
T1 Finish
All Done


In [33]:
thread1 = threading.Thread(target=thread_job,args=("T1",), name='T1')
thread1.start()
thread2 = threading.Thread(target=thread_job_no_sleep,args=("T2",), name='T2')
thread2.start()
thread2.join()
print('All Done')

T1 start
T2 start
T2 Finish
All Done
T1 Finish


In [34]:
thread1 = threading.Thread(target=thread_job,args=("T1",), name='T1')
thread1.start()
thread2 = threading.Thread(target=thread_job_no_sleep,args=("T2",), name='T2')
thread2.start()
thread1.join()
thread2.join()
print('All Done')

T1 start
T2 start
T2 Finish
T1 Finish
All Done
